In [9]:
# =================================================================
# Data Preparation and Feature Engineering
# =================================================================

import pandas as pd
import os

# 1. RAR Extraction and Setup
print("--- Checking RAR extraction ---")
!pip install unrar > /dev/null
!unrar x -o+ "/content/sample_data/Final Project.rar" > /dev/null
print("✅ Extraction setup complete.")

# 2. Load DataFrames
FILE_PATHS = {
    'orders': 'orders_DATA.csv',
    'items': 'Order_items.csv',
    'payments': 'Order_payments.csv',
    'reviews': 'Order_reviews.csv',
    'customers': 'Customers.csv'
}

data_frames = {}

for name, path in FILE_PATHS.items():
    try:
        data_frames[name] = pd.read_csv(path, parse_dates=True, low_memory=False)
        print(f"✅ Loaded {name}.")
    except FileNotFoundError:
        print(f"❌ File not found: {path}.")
    except Exception as e:
        print(f"❌ Error loading {name}: {e}")

if not all(key in data_frames for key in FILE_PATHS.keys()):
    print("❌ Failed to load all required tables.")
    exit()

# 3. Aggregation and Merging

df_orders = data_frames['orders']
df_items = data_frames['items']
df_payments = data_frames['payments']
df_reviews = data_frames['reviews']
df_customers = data_frames['customers']

payments_agg = df_payments.groupby('order_id').agg(Total_Payment_Value=('payment_value', 'sum')).reset_index()
items_agg = df_items.groupby('order_id').agg(Total_Product_Price=('price', 'sum'), Total_Freight_Value=('freight_value', 'sum')).reset_index()
reviews_agg = df_reviews.groupby('order_id').agg(review_score=('review_score', 'max')).reset_index()

df_customers_clean = df_customers[['customer_id', 'customer_unique_id', 'customer_state']].copy()

df_final = df_orders.copy()
df_final = df_final.merge(df_customers_clean, on='customer_id', how='left')
df_final = df_final.merge(payments_agg, on='order_id', how='left')
df_final = df_final.merge(items_agg, on='order_id', how='left')
df_final = df_final.merge(reviews_agg, on='order_id', how='left')

# 4. Feature Engineering (KPIs)

date_cols_to_convert = ['order_purchase_timestamp', 'order_delivered_customer_date', 'order_estimated_delivery_date']
for col in date_cols_to_convert:
    df_final[col] = pd.to_datetime(df_final[col], errors='coerce')

df_final['Total Order Price'] = df_final['Total_Payment_Value']
df_final['Delivery Time (Days)'] = (df_final['order_delivered_customer_date'] - df_final['order_purchase_timestamp']).dt.total_seconds() / (60 * 60 * 24)
df_final['Estimated Delivery Diff (Days)'] = (df_final['order_estimated_delivery_date'] - df_final['order_delivered_customer_date']).dt.total_seconds() / (60 * 60 * 24)
df_final['Purchase Month'] = df_final['order_purchase_timestamp'].dt.to_period('M').astype(str)
df_final['Purchase Day of Week'] = df_final['order_purchase_timestamp'].dt.day_name()

# 5. Column Selection
cols_to_keep = [
    'order_id', 'customer_id', 'customer_unique_id', 'order_status', 'customer_state', 'order_purchase_timestamp',
    'Total Order Price', 'Total_Product_Price', 'Total_Freight_Value',
    'Delivery Time (Days)', 'Estimated Delivery Diff (Days)',
    'Purchase Month', 'Purchase Day of Week', 'review_score',
]

df_final = df_final[cols_to_keep]

print("\n=================================================================")
print("✅ Final DataFrame 'df_final' is ready.")
print("=================================================================")
print("df_final.head():")
print(df_final.head())

--- Checking RAR extraction ---
✅ Extraction setup complete.
✅ Loaded orders.
✅ Loaded items.
✅ Loaded payments.
✅ Loaded reviews.
✅ Loaded customers.

✅ Final DataFrame 'df_final' is ready.
df_final.head():
                           order_id                       customer_id  \
0  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
1  53cdb2fc8bc7dce0b6741e2150273451  b0830fb4747a6c6d20dea0b8c802d7ef   
2  47770eb9100c2d0c44946d9cf07ec65d  41ce2a54c0b03bf3443c3d931a367089   
3  949d5b44dbf5de918fe9c16f97b45f8a  f88197465ea7920adcdbec7375364d82   
4  ad21c59c0840e6cb83a9ceb5573f8159  8ab97904e6daea8866dbdbc4fb7aad2c   

                 customer_unique_id order_status customer_state  \
0  7c396fd4830fd04220f754e42b4e5bff    delivered             SP   
1  af07308b275d755c9edb36a90c618231    delivered             BA   
2  3a653a41f6f9fc3d2a113cf8398680e8    delivered             GO   
3  7c142cf63193a1473d2e66489a9ae977    delivered             RN   
4  72632f0f9dd73df